In [2]:
import requests
import os
import json
from pprint import pprint
import warnings

In [3]:
ZENODO_ACCESS_TOKEN = os.environ.get('ZENODO_ACCESS_TOKEN', '4WUTJLhMlKljZkknoc7llyMcTy3vjBy3I11j0mwUHVSYixxjt9Q72kLo8KMp')
ZENODO_URL = os.environ.get('ZENODO_URL', 'https://sandbox.zenodo.org')
params = {'access_token': ZENODO_ACCESS_TOKEN}
headers = {"Content-Type": "application/json"}

In [6]:
# Create empty record
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions',
  params=params,
  json={},
  headers=headers
)

assert (res.status_code == 201), f'Error creating record: {res.json()}'

newrecord = res.json()
newrecord_deposition_id = newrecord['id']
newrecord_bucket_url = newrecord["links"]["bucket"]
newversion_prereserve_doi = newrecord['metadata']['prereserve_doi']['doi']
pprint(newrecord)

{'conceptrecid': '138825',
 'created': '2024-12-02T20:53:57.302178+00:00',
 'files': [],
 'id': 138826,
 'links': {'badge': 'https://sandbox.zenodo.org/badge/doi/.svg',
           'bucket': 'https://sandbox.zenodo.org/api/files/c9644e81-59d9-4900-95f6-f688c66f44b6',
           'discard': 'https://sandbox.zenodo.org/api/deposit/depositions/138826/actions/discard',
           'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/138826/actions/edit',
           'files': 'https://sandbox.zenodo.org/api/deposit/depositions/138826/files',
           'html': 'https://sandbox.zenodo.org/deposit/138826',
           'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/138826',
           'latest_draft_html': 'https://sandbox.zenodo.org/deposit/138826',
           'newversion': 'https://sandbox.zenodo.org/api/deposit/depositions/138826/actions/newversion',
           'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/138826/actions/publish',
           'registerc

In [17]:
publish_path='publish/1.0'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
zenodo_json

{'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': '<p>This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities.</p><p>The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specification of a standard name, describes the vertical locations corresponding to dimensionless vertical coordinate values, and provides the spatial coordinates of non-rectilinear 

In [18]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newrecord_deposition_id}: {res.json()}'


In [8]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.put(
      f'{newrecord_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'
  #TODO: Verify MD5


In [ ]:
res

In [16]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

{'created': '2024-12-02T20:53:57.302178+00:00',
 'modified': '2024-12-02T21:06:29.686109+00:00',
 'id': 138826,
 'conceptrecid': '138825',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'publication_date': '2003-10-28',
  'description': '<p>This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities.</p><p>The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specifi

In [ ]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

# "New" Version 1.1

In [103]:
#LATEST_DEPOSITION_ID = '129052'
#LATEST_DEPOSITION_ID = '129592'
#LATEST_DEPOSITION_ID = '130529'
LATEST_DEPOSITION_ID = '138826'

# Create new version FROM record
#  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
#  f'{ZENODO_URL}/api/records/{LATEST_DEPOSITION_ID}/versions',
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
  params = params,
  json = {},
  headers = headers
)
if res.status_code != 201:
  if res.status_code == 500 and res.json()['message'] == 'Misconfigured search.':
    warnings.warn(f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}')
  else:
    assert (res.status_code == 201), f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}'

In [104]:
# TO BE KEEP in case temporla issues appear again
#TEMP_ID = 129747
#res = requests.get(
#  f'{ZENODO_URL}/api/deposit/depositions/{TEMP_ID}',
#  params = params,
#  headers = headers
#)
#assert (res.status_code == 200), f'Error retriving new verions deposition {TEMP_ID}: {res.json()}'

newversion_record = res.json()
newversion_deposition_id = newversion_record['id']
newversion_bucket_url = newversion_record['links']['bucket']
newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
newversion_record

{'created': '2024-12-02T21:48:19.073455+00:00',
 'modified': '2024-12-02T21:48:19.220533+00:00',
 'id': 138838,
 'conceptrecid': '138825',
 'conceptdoi': '10.5072/zenodo.138825',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': '<p>This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities.</p><p>The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via spe

In [110]:
#delete files
for file in newversion_record['files']:
  res = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  assert (res.status_code == 204), f'Error deleting file {file['filename']}: {res.json()}'


AssertionError: Error deleting file conformance.pdf: {'status': 404, 'message': "Record '138838' has no file 'conformance.pdf'."}

In [111]:
publish_path='publish/1.12'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
#zenodo_json

In [112]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newversion_deposition_id}: {res.json()}'


In [ ]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.psut(
      f'{newversion_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'


## BE CAREFUL THIS WILL PUBLISH THE NEW VERSION

In [102]:
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/actions/publish',
  params = params,
  json = {},
  headers = headers
)
assert (res.status_code == 202), f'Error publishing deposition id: {newversion_deposition_id}'
res.json()

{'created': '2024-12-02T21:45:18.275348+00:00',
 'modified': '2024-12-02T21:45:18.609023+00:00',
 'id': 138837,
 'conceptrecid': '138825',
 'doi': '10.5072/zenodo.138837',
 'conceptdoi': '10.5072/zenodo.138825',
 'doi_url': 'https://doi.org/10.5072/zenodo.138837',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'doi': '10.5072/zenodo.138837',
  'publication_date': '2023-12-05',
  'description': '<p>This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities.</p><p>

# Parking Code

In [ ]:
#Read metadata
with open('publish/1.11/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
r.status_code

In [ ]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.11/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

# New Version 1.12-rc7

In [ ]:
LATEST_DEPOSITION_ID = '129052'
# Create new record
r = requests.post(f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
                   params=params,
                   json={},
                   headers=headers)
# 201
if r.status_code == 201:
  newversion_record = r.json()
  newversion_deposition_id = newversion_record['id']
  newversion_bucket_url = newversion_record['links']['bucket']
  newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
else:
  raise requests.RequestException(r.json())



In [ ]:
#delete files
for file in newversion_record['files']:
  r = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  if r.status_code != 204:
    r.json()

In [ ]:
#Read metadata
with open('publish/1.12-rc7/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)


In [ ]:
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
if r.status_code != 200:
    r.json()

In [ ]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.12-rc7/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

In [ ]:
newversion_record

# File checksum

In [ ]:
import os
import hashlib

def calculate_md5(file_path):
    """Calculate MD5 checksum for a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def list_files_and_generate_metadata():
    """List PDF and HTML files in the current directory and generate metadata."""
    files_metadata = []
    for file in os.listdir("."):
        if file.endswith(".pdf") or file.endswith(".html"):
            size = os.path.getsize(file)
            checksum = calculate_md5(file)
            files_metadata.append((file, size, checksum))

    # Find maximum lengths for formatting
    max_filename_len = max(len(file[0]) for file in files_metadata) + 2
    max_size_len = max(len(str(file[1])) for file in files_metadata) + 2
    max_checksum_len = len("checksum") + 2

    # Generate formatted output
    print("Files Section:")
    print(" " * 4 + "{" + ",\n    ".join(
        f'{{"filename": {file[0]:<{max_filename_len}} "size": {file[1]:<{max_size_len}} "checksum": "md5:{file[2]:<{max_checksum_len}}"}}'
        for file in files_metadata
    ) + "}")
    print()

if __name__ == "__main__":
    list_files_and_generate_metadata()